In [5]:
'''
--------------------------------------------------------------------------------
Author: Max White
Language: Python3
--------------------------------------------------------------------------------
Catherine Bradshaw has already written a script to request the data for you. 
This code is used to merge it all together

INSTRUCTION 1: Open 'basic_info.py' and update the information in there. The following code will then work.
INSTRUCTION 2: Change the minimum_weight below to see how it affects the masking of the data.
    Reducing the weight will include more data; increasing the weight will exclude more data.
'''
import importlib # This is used to reload the basic_info script in case you update it
import basic_info # Importing the basic information script you need to update!
import shapefile_extraction # Importing the shapefile extraction script
import os
import geopandas as gpd
import iris
from iris.util import equalise_attributes
import iris.quickplot as qplt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
importlib.reload(basic_info)

directory_path = basic_info.directory_filepath
data_folder = basic_info.data_folder
tas_files = basic_info.tas_files
td_files = basic_info.td_files
shapefile_folder = basic_info.shapefile_folder
shapefile_file = basic_info.shapefile_file
shapefile_extraction.create_my_shape_outline(shapefile_file)

## Concatenation

In [ ]:
def concatenate_data(filepath, cube_name):
    """
    Load the data from the specified file path and concatenate the cubes into a single cube for further processing.
    """
    cubelist = iris.cube.CubeList([])
    files = os.listdir(filepath)
    for file in files:
        if file.endswith('.nc'):
            cubelist.append(iris.load_cube(f'{filepath}/{file}')) # So far the data is loaded into a list of cubes but not concatenated
    
    # Sort cubes by time coordinate
    cubelist.sort(key=lambda cube: cube.coord('time').points[0]) # This line sorts the cubes by the time coordinate so that they can be concatenated in the correct order
    
    # Remove overlapping cubes
    non_overlapping_cubelist = iris.cube.CubeList([])
    last_time = None
    for cube in cubelist:
        current_time = cube.coord('time').points[0]
        if last_time is None or current_time > last_time:
            non_overlapping_cubelist.append(cube)
            last_time = cube.coord('time').points[-1]
    
    removed_attributes = equalise_attributes(non_overlapping_cubelist)
    concatenated_cube = non_overlapping_cubelist.concatenate_cube()
    return concatenated_cube

tas = concatenate_data(tas_files, 'tas')
td = concatenate_data(td_files, 'td')

importlib.reload(basic_info)
shapefile_outline = gpd.read_file(basic_info.shapefile_outline)

## Masking using a shapefile

In [ ]:
def save_and_mask_cube_for_later_use(cube, name, shapefile_outline, filepath):
    """
    Save the cube to a netCDF file for later use.

    Parameters:
    -----------
    cube: iris.cube.Cube
        The cube to save.
    name: str
        The name of the cube.
    shapefile_outline: str
        The file path to the shapefile outline.
    filepath: str
        The file path to save the cube to.
    """
    masked_cube = iris.util.mask_cube_from_shapefile(cube, shapefile_outline, minimum_weight=0.5)
    iris.save(masked_cube, filepath)
    f = open('basic_info.py', 'a')
    f.write(f"\n{name}_filepath = '{data_folder}/{name}_cube.nc'\n")
    f.close()
    print('shapefile created and saved as outline.shp')
    
save_and_mask_cube_for_later_use(tas, 'tas', shapefile_outline.geometry[0], f'{data_folder}/tas_cube.nc')
save_and_mask_cube_for_later_use(td, 'td', shapefile_outline.geometry[0], f'{data_folder}/td_cube.nc')

tas_cube_masked = iris.util.mask_cube_from_shapefile(tas, shapefile_outline.geometry[0], minimum_weight=0.5)

td_cube_masked = iris.util.mask_cube_from_shapefile(td, shapefile_outline.geometry[0], minimum_weight=0.5)

# Plotting the masked tas_cube with shapefile overlayed to check weighting
fig = plt.figure(figsize=[10, 10])
ax = plt.axes(projection=ccrs.PlateCarree())
qplt.pcolormesh(tas_cube_masked[0], cmap='viridis')

# Overlay the shapefile
shapefile_outline.boundary.plot(ax=ax, edgecolor='red')

plt.show()